# Carga no Banco de Dados - SQLite

Este notebook é responsável por **carregar os dados da camada Gold em um banco de dados SQLite**.

## Objetivos:
- Conectar ao banco de dados SQLite
- Carregar tabelas dimensionais e fatos no banco
- Carregar métricas agregadas
- Validar o carregamento verificando as tabelas criadas

## Resultado:
Um banco de dados estruturado e pronto para consultas SQL e análises.

In [1]:
import pandas as pd
import sqlite3 as sql

# Conectar ao banco SQLite3
conn = sql.connect('data/pipeline.db')
print("Conexão com banco estabelecida")

# ==========================================
#          CARREGAR TABELAS (GOLD)
# ==========================================
country = pd.read_parquet('data/gold/dim_country.parquet')
country.to_sql('dim_country', conn, if_exists='replace', index=False)

customer = pd.read_parquet('data/gold/dim_customer.parquet')
customer.to_sql('dim_customer', conn, if_exists='replace', index=False)

date = pd.read_parquet('data/gold/dim_date.parquet') 
date.to_sql('dim_date', conn, if_exists='replace', index=False)

product = pd.read_parquet('data/gold/dim_product.parquet')
product.to_sql('dim_product', conn, if_exists='replace', index=False) 

fact = pd.read_parquet('data/gold/fact_all.parquet')
fact.to_sql('fact', conn, if_exists='replace', index=False)


# ==========================================
#          CARREGAR DADOS AGREGADOS (GOLD)
# ==========================================
metrics = pd.read_parquet('data/gold/metrics.parquet')
metrics.to_sql('metrics', conn, if_exists='replace', index=False)


# ==========================================
#     VERIFICAR TABELAS CRIADAS
# =========================================
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()
print("\nTabelas no banco de dados:")

for tabela in tabelas:
    print(f"  - {tabela[0]}")

conn.close()
print("\nDados carregados com sucesso no banco de dados!")

Conexão com banco estabelecida

Tabelas no banco de dados:
  - dim_country
  - dim_customer
  - dim_date
  - dim_product
  - fact
  - metrics

Dados carregados com sucesso no banco de dados!

Tabelas no banco de dados:
  - dim_country
  - dim_customer
  - dim_date
  - dim_product
  - fact
  - metrics

Dados carregados com sucesso no banco de dados!


### Carregamento das Tabelas

Este código:
1. Estabelece conexão com o banco SQLite (cria o arquivo se não existir)
2. Lê cada arquivo Parquet da camada Gold
3. Carrega cada tabela no banco usando `to_sql()`
4. Substitui tabelas existentes se já houver dados anteriores
5. Lista todas as tabelas criadas para validação

As tabelas carregadas incluem dimensões (país, cliente, data, produto), a tabela fato unificada e as métricas agregadas.

In [2]:
# Consulta SQL para ver o total de vendas na tabela fact
conn = sql.connect('data/pipeline.db')
query = "SELECT SUM(total_value) AS total_vendas FROM fact;"
total_vendas = pd.read_sql_query(query, conn)
print(f"Total de vendas: {total_vendas.iloc[0,0]:,.2f}")
conn.close()

Total de vendas: 9,759,687.46


### Validação do Carregamento

Executamos uma consulta SQL simples para verificar se os dados foram carregados corretamente, calculando o total de vendas diretamente no banco de dados.